In [52]:
import pandas as pd
import glob
import re
import numpy as np

# ----------------------------------------------------------------------
# 1. Definición de la lista de Capitales/Provincias de España
# ----------------------------------------------------------------------
CAPITALES_A_PROVINCIAS = {
    "MADRID": "Madrid",
    "BARCELONA": "Barcelona",
    "VALENCIA/VALÈNCIA": "Valencia/València",
    "SEVILLA": "Sevilla",
    "ZARAGOZA": "Zaragoza",
    "MÁLAGA": "Málaga",
    "MURCIA": "Murcia",
    "PALMA DE MALLORCA": "Balears, Illes",
    "LAS PALMAS DE GRAN CANARIA": "Palmas, Las",
    "ALACANT/ALICANTE": "Alicante/Alacant",
    "BILBAO": "Bizkaia",
    "CÓRDOBA": "Córdoba",
    "VALLADOLID": "Valladolid",
    "VITORIA-GASTEIZ": "Araba/Álava",
    "A CORUÑA": "Coruña, A",
    "GRANADA": "Granada",
    "OVIEDO": "Asturias",
    "SANTA CRUZ DE TENERIFE": "Santa Cruz de Tenerife",
    "PAMPLONA/IRUÑA": "Navarra",
    "ALMERÍA": "Almería",
    "DONOSTIA/SAN SEBASTIÁN": "Gipuzkoa",
    "BURGOS": "Burgos",
    "ALBACETE": "Albacete",
    "SANTANDER": "Cantabria",
    "CASTELLÓ DE LA PLANA": "Castellón/Castelló",
    "LOGROÑO": "Rioja, La",
    "BADAJOZ": "Badajoz",
    "SALAMANCA": "Salamanca",
    "HUELVA": "Huelva",
    "LLEIDA": "Lleida",
    "TARRAGONA": "Tarragona",
    "LEÓN": "León",
    "JAÉN": "Jaén",
    "CÁDIZ": "Cádiz",
    "OURENSE": "Ourense",
    "GIRONA": "Girona",
    "LUGO": "Lugo",
    "CACERES": "Cáceres",
    "GUADALAJARA": "Guadalajara",
    "MELILLA": "Melilla",
    "TOLEDO": "Toledo",
    "CEUTA": "Ceuta",
    "PONTEVEDRA": "Pontevedra",
    "PALENCIA": "Palencia",
    "CIUDAD REAL": "Ciudad Real",
    "ZAMORA": "Zamora",
    "ÁVILA": "Ávila",
    "CUENCA": "Cuenca",
    "HUESCA": "Huesca",
    "SEGOVIA": "Segovia",
    "SORIA": "Soria",
    "TERUEL": "Teruel"
}

# ----------------------------------------------------------------------
# 2. Función de normalización de textos
# ----------------------------------------------------------------------
def normalizar_texto(texto):
    if pd.isna(texto):
        return ''
    texto = str(texto).strip().upper()
    reemplazos = {
        'Á':'A','À':'A',
        'É':'E','È':'E',
        'Í':'I','Ì':'I',
        'Ó':'O','Ò':'O',
        'Ú':'U','Ù':'U','Ü':'U',
        'Ñ':'N','Ç':'C'
    }
    for orig, nuevo in reemplazos.items():
        texto = texto.replace(orig, nuevo)
    return texto

# ----------------------------------------------------------------------
# 3. Función de procesamiento para cada CSV
# ----------------------------------------------------------------------
def procesar_csv(file_path):
    df_temp = pd.read_csv(file_path, sep=";", encoding="utf-8", low_memory=False)
    COL_MUNICIPAL = df_temp.columns[0]

    # Extracción de ID y Municipio
    patron = r"(\d+)\s(.*)"
    df_temp[['ID municipio', 'Municipio']] = (
        df_temp[COL_MUNICIPAL].astype(str).str.extract(patron, expand=True)
    )
    df_temp['Municipio'] = df_temp['Municipio'].str.strip()
    df_temp['ID municipio'] = pd.to_numeric(df_temp['ID municipio'], errors='coerce')

    # Normalizar municipios para la comparación
    municipios_normalizados = df_temp['Municipio'].apply(normalizar_texto).tolist()

    provincia_encontrada = np.nan
    for capital, provincia in CAPITALES_A_PROVINCIAS.items():
        provincia_norm = normalizar_texto(provincia)
        if provincia_norm in municipios_normalizados:
            provincia_encontrada = provincia
            break

    df_temp['Provincia'] = provincia_encontrada
    df_temp = df_temp.drop(columns=[COL_MUNICIPAL])
    return df_temp

# ----------------------------------------------------------------------
# 4. Ejecución del Script Principal
# ----------------------------------------------------------------------
ruta = "../poblacion por municipios crudo/*.csv"
archivos = glob.glob(ruta)

print(f"Ruta utilizada para la búsqueda: {ruta}")
print(f"Se encontraron {len(archivos)} archivos CSV.")

if not archivos:
    print("\n⚠️ ¡ADVERTENCIA! No se encontraron archivos. Revisa la ruta.")
else:
    try:
        df = pd.concat((procesar_csv(f) for f in archivos), ignore_index=True)
    except Exception as e:
        print(f"\n❌ Error durante la lectura o concatenación: {e}")


Ruta utilizada para la búsqueda: ../poblacion por municipios crudo/*.csv
Se encontraron 52 archivos CSV.


In [55]:
import os
os.makedirs("../datos combinados", exist_ok=True)
df.to_csv("../datos combinados/Poblacion_por_municipios.csv", index=False)